# Before running this script
This jpynb is an example tutorial that only tested for running on ```ivysaur```, the ESCAL's machine. \
Also, ```gengar``` machine is actually used to run this project. Additional ssh setup is required as well as command modification as shown below.

## 1. To avoid using ```sudo```(since no-interactive mode here):
A quick fix on host(gengar) before running this tutorial:
```sudo chmod 666 /var/run/docker.sock``` \
Reference: https://stackoverflow.com/questions/48957195/how-to-fix-docker-got-permission-denied-issue/51362528#51362528

## 2. Under ESCAL's environment setup, to avoid interactive password key in:
On ivysaur - ```cat id_rsa.pub >> authorized_keys``` \
This tutorial using account ```khsu037``` and machine ```gengar``` as example. \
Reference: 
https://unix.stackexchange.com/questions/612023/how-to-set-passwordless-authentication-in-a-cluster-where-users-home-directory

## Setup docker environment on Gengar
This section builds the docker image and the container that this project is based on.

In [1]:
! ssh khsu037@gengar \
"lsb_release -a ; \
cd GPTPU.internal ; \
sh scripts/docker_setup.sh"

Distributor ID:	Ubuntu
Description:	Ubuntu 16.04.7 LTS
Release:	16.04
Codename:	xenial
No LSB modules are available.
[gptpu_new] - building docker image from dockerfile...
Sending build context to Docker daemon  42.32MB
Step 1/18 : FROM tensorflow/tensorflow:2.9.2 AS BUILD
 ---> cfca955962ea
Step 2/18 : RUN ln -snf /usr/share/zoneinfo/$CONTAINER_TIMEZONE /etc/localtime && echo $CONTAINER_TIMEZONE > /etc/timezone &&     apt-get update &&     apt-get install -y tzdata make vim cmake g++ git python3 curl unzip zip libusb-1.0-0
 ---> Using cache
 ---> 07d4edbeded9
Step 3/18 : FROM BUILD AS BUILD1
 ---> 07d4edbeded9
Step 4/18 : COPY update_sources.sh /
 ---> Using cache
 ---> 1b21581ce951
Step 5/18 : RUN /update_sources.sh
 ---> Using cache
 ---> a0bb95f26602
Step 6/18 : RUN dpkg --add-architecture armhf
 ---> Using cache
 ---> 8810296f0b7c
Step 7/18 : RUN dpkg --add-architecture arm64
 ---> Using cache
 ---> 01cad329f9e2
Step 8/18 : RUN apt-get update     && DEBIAN_FRONTEND=noninteractive 

## Build project
This section calls project building script ```build.sh``` to compile. It may takes a few minutes since tensorflow repo will be downloaded during make process. \
TODO: avoid re-downloading tf repo to save project build time.

In [2]:
! ssh khsu037@gengar \
"cd GPTPU.internal ; \
docker exec gptpu_new_container sh -c \"sh ./scripts/build.sh 2> build_log.txt\""

-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Configuring done
-- Generating done
-- Build files have been written to: /home/build
Scanning dependencies of target tf
[  5%] Creating directories for 'tf'
[ 11%] Performing download step (git clone) for 'tf'
[ 16%] No patch step for 'tf'
[ 22%] Performing update step for 'tf'
[ 27%] Performing configure step for 'tf'
checking sha256 of tensorflow/lite/tools/make/downloads/eigen
/tmp/tmp.92QRyUPRIT/

## Test run

In [3]:
! ssh khsu037@gengar \
"cd GPTPU.internal ; \
docker exec gptpu_new_container sh -c \"./out/k8/minimal ./model/mobilenet_v2_1.0_224_quant_edgetpu.tflite 1048576 1024 1\""

enumerated edgetpu: 
/dev/apex_2
/dev/apex_0
/dev/apex_3
/dev/apex_1
opened device: type: Apex (PCIe), path: /dev/apex_2
zero_point: 0, scale: 0.00390625
input[0]: 103, raw output[0]: 0
input[1]: 198, raw output[1]: 0
input[2]: 105, raw output[2]: 0
input[3]: 115, raw output[3]: 0
input[4]: 81, raw output[4]: 0
input[5]: 255, raw output[5]: 1
input[6]: 74, raw output[6]: 0
input[7]: 236, raw output[7]: 0
input[8]: 41, raw output[8]: 0
input[9]: 205, raw output[9]: 0
kernel time: 1.1032 (ms), average over 1 time(s).
